In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import csv


In [ ]:
# import training data
from sklearn import preprocessing


def import_training_labels():
    labels = []
    with open('train.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            labels.append(row[1])

    labels.pop(0)

    return labels

label_list = ["blues", "hiphop", "jazz", "metal", "disco", "country", "rock", "reggae", "classical", "pop"]

training_data = np.load("Training_Data_vector.npy").astype(np.double)
tensor_train_data = torch.from_numpy(training_data)

training_labels = import_training_labels()
le = preprocessing.LabelEncoder() # Changes text labels to numerical tensors
le.fit(label_list)
new_labels = le.transform(training_labels)
tensor_labels = torch.tensor(new_labels, dtype=torch.double)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


entries_per_sample = 24


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.full_connect1 = nn.Linear(entries_per_sample, 256)
        self.full_connect2 = nn.Linear(256, 128)
        self.full_connect3 = nn.Linear(128, 64)
        self.full_connect4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.full_connect1(x))
        x = F.relu(self.full_connect2(x))
        x = F.relu(self.full_connect3(x))
        x = F.log_softmax(self.full_connect4(x), dim=0)
#         x = self.full_connect4(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(tensor_labels)):
        # get the inputs; data is a list of [inputs, labels]
        inputs = tensor_train_data[:,i]
        label = tensor_labels[i]
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        print(outputs)
        print(label)
#         loss = criterion(outputs, label)
        loss = F.nll_loss(outputs, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = './nn_classfier1.pth'
torch.save(net.state_dict(), PATH)